In [ ]:
import os
from google.colab import files
files.upload()  # Pilih file `kaggle.json`
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Beri izin agar hanya bisa dibaca oleh user

Saving kaggle.json to kaggle.json


In [ ]:
import os
from google.colab import files
!pip install kaggle
!kaggle datasets download -d rafliindrawan/acne-dataset
acne_dir = os.path.join("/content/merged_dataset", "Acne", "data")
os.makedirs(acne_dir, exist_ok=True)
!unzip acne-dataset.zip -d merged_dataset/Acne/data

Dataset URL: https://www.kaggle.com/datasets/rafliindrawan/acne-dataset
License(s): unknown
Archive:  acne-dataset.zip
  inflating: merged_dataset/Acne/data/07Acne0811011 - Copy_acne_0.jpg  
  inflating: merged_dataset/Acne/data/07Acne0811011_acne_0.jpg  
  inflating: merged_dataset/Acne/data/07AcnePittedScars_acne_0.jpg  
  inflating: merged_dataset/Acne/data/07RosaceaOK08280631 - Copy_acne_1.jpg  
  inflating: merged_dataset/Acne/data/07RosaceaOK08280631_acne_1.jpg  
  inflating: merged_dataset/Acne/data/07SteroidPerioral11 - Copy_acne_2.jpg  
  inflating: merged_dataset/Acne/data/07SteroidPerioral11_acne_2.jpg  
  inflating: merged_dataset/Acne/data/0_Skin_acne_0.jpg  
  inflating: merged_dataset/Acne/data/0_before_acne_0.jpg  
  inflating: merged_dataset/Acne/data/10_Skin11_acne_0.jpg  
  inflating: merged_dataset/Acne/data/10_Skin1_acne_0.jpg  
  inflating: merged_dataset/Acne/data/10_acne_0.jpg  
  inflating: merged_dataset/Acne/data/11118622-6820319-image-a-24_15528711043151_acn

In [ ]:
%cd /content
!git clone https://github.com/Fliw/Progressive-GAN-pytorch.git
%cd Progressive-GAN-pytorch

/content
Cloning into 'Progressive-GAN-pytorch'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 95 (delta 50), reused 7 (delta 7), pack-reused 16 (from 1)
Receiving objects: 100% (95/95), 34.27 KiB | 548.00 KiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Progressive-GAN-pytorch


In [ ]:
!python train.py --path /content/merged_dataset/Acne --init_step 6 --checkpoint "/content/model"

Loading checkpoints from /content/model...
Loading checkpoints from /content/model...
Optimizers loaded successfully!
  1% 499/50000 [04:14<6:57:02,  1.98it/s]500; G: 1.315; D: 1.543; Grad: 1.461; Alpha: 0.020
  2% 999/50000 [08:27<6:53:34,  1.97it/s]1000; G: 7.876; D: 2.738; Grad: 0.805; Alpha: 0.040
  2% 1039/50000 [08:49<6:55:49,  1.96it/s]
Traceback (most recent call last):
  File "/content/Progressive-GAN-pytorch/train.py", line 281, in <module>
    train(generator, discriminator, args.init_step, loader, args.total_iter, args.start_iter)
  File "/content/Progressive-GAN-pytorch/train.py", line 118, in train
    real_image = real_image.to(device)
                 ^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


In [ ]:
# 1. Copot pydantic & klien Gradio bermasalah
!pip uninstall -y pydantic gradio_client

# 2. Pasang versi kompatibel
!pip install "pydantic==2.10.6" "gradio>=4.17.0" "gradio_client>=0.13.0"

# 3. RESTART runtime / kernel Python


Found existing installation: pydantic 2.10.6
Uninstalling pydantic-2.10.6:
  Successfully uninstalled pydantic-2.10.6
Found existing installation: gradio_client 1.8.0
Uninstalling gradio_client-1.8.0:
  Successfully uninstalled gradio_client-1.8.0
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached gradio_client-1.8.0-py3-none-any.whl.metadata (7.1 kB)
Using cached pydantic-2.10.6-py3-none-any.whl (431 kB)
Using cached gradio_client-1.8.0-py3-none-any.whl (322 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.10.0 requires websockets<15.1.0,>=13.0.0, but you have websockets 12.0 which is incompatible.


In [ ]:
%cd /content/Progressive-GAN-pytorch

/content/Progressive-GAN-pytorch


In [ ]:
import torch, os, gradio as gr, numpy as np
from torchvision import utils, transforms
from progan_modules import Generator

# ====== konfigurasi tetap ======
CHECKPOINT_DIR = "/content/model"
Z_DIM, CHANNEL_SIZE = 128, 128
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
FIXED_STEP  = 6        # 256×256
FIXED_ALPHA = 0.0      # sesuai slider di screenshot
# ==============================

g_running = Generator(CHANNEL_SIZE, Z_DIM, pixel_norm=False, tanh=False).to(DEVICE)
g_running.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, "g.model"), map_location=DEVICE))
g_running.eval()

to_pil = transforms.ToPILImage()

@torch.inference_mode()
def sample_images(n_images: int = 50,
                  seed: int | None = None):
    if seed is not None and seed >= 0:
        torch.manual_seed(seed); np.random.seed(seed)
    else:
        torch.seed()

    z = torch.randn(n_images, Z_DIM, device=DEVICE)
    imgs = g_running(z, step=FIXED_STEP, alpha=FIXED_ALPHA).cpu()

    grid = utils.make_grid(imgs, nrow=10, normalize=True, value_range=(-1, 1))
    return to_pil(grid)

### Gradio UI – hanya 2 input (Jumlah gambar & Seed)
demo = gr.Interface(
    fn=sample_images,
    inputs=[
        gr.Slider(1, 200, value=50, step=10, label="Jumlah Gambar (kelipatan 10)"),
        gr.Number(value=-1, precision=0, label="Seed (‑1 = acak)"),
    ],
    outputs=gr.Image(type="pil", label="Grid Hasil"),
    title="Progressive Growing Generative Adversarial Network",
    description="contoh implementasi PGGAN untuk dataset jerawat",
    allow_flagging="never",
)

if __name__ == "__main__":
    demo.queue()
    demo.launch(show_api=False, share=True)


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4112175f2aaeb31f14.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
